This notebook is developed using the `Python 3 (Data Science)` kernel on an `ml.t3.medium` instance.

In [ ]:
!pip install -q sagemaker-experiments

In [ ]:
import sagemaker
import json
import boto3

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = 'sagemaker-studio-book/chapter07'

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
source_file='s3://sagemaker-sample-files/datasets/tabular/synthetic/churn.txt'
local_prefix='churn_data'
os.makedirs(local_prefix, exist_ok=True)
sagemaker.s3.S3Downloader.download(source_file, local_prefix)

In [ ]:
df=pd.read_csv(f'./{local_prefix}/churn.txt')
df['CustomerID']=df.index
df.head()

In [ ]:
df.columns

In [ ]:
df[["Int'l Plan", "VMail Plan"]] = df[["Int'l Plan", "VMail Plan"]].replace(to_replace=['yes', 'no'], value=[1, 0])

In [ ]:
df['Churn?'] = df['Churn?'].replace(to_replace=['True.', 'False.'], value=[1, 0])

In [ ]:
columns=['Churn?', 'State', 'Account Length', "Int'l Plan",
           'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge',
           'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
           'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
           'CustServ Calls']
df.index = df['CustomerID']
df_processed = df[columns]

In [ ]:
df_processed.head()

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_processed, test_size=0.1, random_state=42, 
                                     shuffle=True, stratify=df_processed['State'])

In [ ]:
columns_no_target=['Account Length', "Int'l Plan", 'VMail Plan', 'VMail Message', 'Day Mins',
                   'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
                   'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge', 'CustServ Calls']

df_test.to_csv(f'{local_prefix}/churn_test.csv')
df_test[columns_no_target].to_csv(f'{local_prefix}/churn_test_no_target.csv', 
                                  index=False)

sagemaker.s3.S3Uploader.upload(f'{local_prefix}/churn_test.csv', 
                               f's3://{bucket}/{prefix}/{local_prefix}')
sagemaker.s3.S3Uploader.upload(f'{local_prefix}/churn_test_no_target.csv', 
                               f's3://{bucket}/{prefix}/{local_prefix}')

In [ ]:
from sagemaker.amazon.amazon_estimator import image_uris
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from botocore.exceptions import ClientError
import time
from time import gmtime, strftime

image = image_uris.retrieve(region=region, framework='xgboost', version='1.3-1')
train_instance_type = 'ml.m5.xlarge'
train_instance_count = 1
s3_output = f's3://{bucket}/{prefix}/{local_prefix}/training'

experiment_name = 'churn-prediction'

try:
    experiment = Experiment.create(
        experiment_name=experiment_name, 
        description='Training churn prediction models based on telco churn dataset.')
except ClientError as e:
    print(f'{experiment_name} experiment already exists! Reusing the existing experiment.')
    

def launch_training_job(state, train_data_s3, val_data_s3):
    exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
    jobname = f'churn-xgb-{state}-{exp_datetime}'

    # Creating a new trial for the experiment
    exp_trial = Trial.create(experiment_name=experiment_name, 
                             trial_name=jobname)

    experiment_config={
        'ExperimentName': experiment_name,
        'TrialName': exp_trial.trial_name,
        'TrialComponentDisplayName': 'Training'}

    xgb = sagemaker.estimator.Estimator(image,
                                        role,
                                        instance_count=train_instance_count,
                                        instance_type=train_instance_type,
                                        output_path=s3_output,
                                        enable_sagemaker_metrics=True,
                                        sagemaker_session=sess)
    xgb.set_hyperparameters(objective='binary:logistic', num_round=20)
    
    train_input = sagemaker.inputs.TrainingInput(s3_data=train_data_s3, 
                                                 content_type='csv')
    val_input = sagemaker.inputs.TrainingInput(s3_data=val_data_s3, 
                                               content_type='csv')
    data_channels={'train': train_input, 'validation': val_input}
    
    xgb.fit(inputs=data_channels, job_name=jobname, 
            experiment_config=experiment_config, wait=False)

    return xgb

In [ ]:
dict_estimator = {}
for state in df_processed.State.unique()[:5]:
    print(state)
    output_dir = f's3://{bucket}/{prefix}/{local_prefix}/by_state'
    df_state = df_train[df_train['State']==state].drop(labels='State', axis=1)
    df_state_train, df_state_val = train_test_split(df_state, test_size=0.1, random_state=42, 
                                                    shuffle=True, stratify=df_state['Churn?'])
    
    df_state_train.to_csv(f'{local_prefix}/churn_{state}_train.csv', index=False)
    df_state_val.to_csv(f'{local_prefix}/churn_{state}_val.csv', index=False)
    sagemaker.s3.S3Uploader.upload(f'{local_prefix}/churn_{state}_train.csv', output_dir)
    sagemaker.s3.S3Uploader.upload(f'{local_prefix}/churn_{state}_val.csv', output_dir)
    
    dict_estimator[state] = launch_training_job(state, out_train_csv_s3, out_val_csv_s3)
    time.sleep(2)

In [ ]:
def wait_for_training_job_to_complete(estimator):
    job = estimator.latest_training_job.job_name
    print(f"Waiting for job: {job}")
    status = estimator.latest_training_job.describe()["TrainingJobStatus"]
    while status == "InProgress":
        time.sleep(45)
        status = estimator.latest_training_job.describe()["TrainingJobStatus"]
        if status == "InProgress":
            print(f"{job} job status: {status}")
    print(f"DONE. Status for {job} is {status}\n")

In [ ]:
for est in list(dict_estimator.values()):
    wait_for_training_job_to_complete(est)

In [ ]:
from sagemaker.multidatamodel import MultiDataModel
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

model_PA = dict_estimator['PA'].create_model(role=role, image_uri=image)
model_data_prefix = f's3://{bucket}/{prefix}/{local_prefix}/multi_model_artifacts/'
exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
model_name = f'churn-xgb-mme-{exp_datetime}'
endpoint_name = model_name
hosting_instance_type = 'ml.c5.xlarge'
hosting_instance_count = 1

mme = MultiDataModel(name=model_name,
                     model_data_prefix=model_data_prefix,
                     model=model_PA,  # passing our model - passes container image needed for the endpoint
                     sagemaker_session=sess)
predictor = mme.deploy(initial_instance_count=hosting_instance_count, 
                       instance_type=hosting_instance_type, 
                       endpoint_name=endpoint_name,
                       serializer = CSVSerializer(),
                       deserializer = JSONDeserializer()))

In [ ]:
list(mme.list_models())

In [ ]:
for state, est in dict_estimator.items():
    artifact_path = est.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
    model_name = f'{state}.tar.gz'
    print(model_name)
    # This is copying over the model artifact to the S3 location for the MME.
    mme.add_model(model_data_source=artifact_path, model_data_path=model_name)

In [ ]:
list(mme.list_models())

In [ ]:
def sample_test_data(state):
    sample = df_test[df_test['State']==state].sample(1)
    sample[["Int'l Plan", 'VMail Plan']]=sample[["Int'l Plan", 'VMail Plan']].astype(int)
    target = sample['Churn?'].values[0].tolist()
    sample = sample.values[0][2:].tolist()    

    return sample, target

In [ ]:
start_time = time.time()

state='PA'
test_data=sample_test_data(state)
print(test_data[0])
prediction = predictor.predict(data=test_data[0], 
                               target_model=f'{state}.tar.gz')

duration = time.time() - start_time
print(f'{prediction} vs ground truth {test_data[1]}')
print('It took {:,d} ms\n'.format(int(duration * 1000)))

In [ ]:
start_time = time.time()

state='SC'
test_data=sample_test_data(state)
print(test_data[0])
prediction = predictor.predict(data=test_data[0], 
                               target_model=f'{state}.tar.gz')

duration = time.time() - start_time
print(f'{prediction} vs ground truth {test_data[1]}')
print('It took {:,d} ms\n'.format(int(duration * 1000)))

In [ ]:
start_time = time.time()

state='VA'
test_data=sample_test_data(state)
print(test_data[0])
prediction = predictor.predict(data=test_data[0], 
                               target_model=f'{state}.tar.gz')

duration = time.time() - start_time
print(f'{prediction} vs ground truth {test_data[1]}')
print('It took {:,d} ms\n'.format(int(duration * 1000)))

Uncomment and run the next cell to delete endpoints to stop incurring cost.

In [ ]:
# predictor.delete_endpoint()